In [43]:
import pandas as pd

In [44]:
ah_df = pd.read_csv('../data/clean/all_housing.csv')
ah_df

,OWN_OCC,LU_DESC,UNIT_NUM,ST_NUM,ST_NAME,ZIPCODE,CITY,RES_UNITS
0,Y,THREE-FAM DWELLING,NaN,104.0,PUTNAM ST,2128,EAST BOSTON,0
1,Y,THREE-FAM DWELLING,NaN,197.0,LEXINGTON ST,2128,EAST BOSTON,0
2,N,THREE-FAM DWELLING,NaN,199.0,LEXINGTON ST,2128,EAST BOSTON,0
3,N,THREE-FAM DWELLING,NaN,201.0,LEXINGTON ST,2128,EAST BOSTON,0
4,Y,TWO-FAM DWELLING,NaN,203.0,LEXINGTON ST,2128,EAST BOSTON,0
...,...,...,...,...,...,...,...,...
275405,N,STRIP CTR STORES,NaN,2198.0,COMMONWEALTH AV,2135,BRIGHTON,0
275406,N,STRIP CTR STORES,NaN,2199.0,COMMONWEALTH AV,2135,BRIGHTON,0
275407,N,STRIP CTR STORES,NaN,2200.0,COMMONWEALTH AV,2135,BRIGHTON,0
275408,N,STRIP CTR STORES,NaN,2201.0,COMMONWEALTH AV,2135,BRIGHTON,0


In [45]:
ir_df = pd.read_csv('../data/clean/income_restricted_housing.csv')
ir_df

,PROJ_NAME,ZIPCODE,SECTION8,RES_UNITS,ST_NUM,ST_NAME,CITY
0,Abbot Street/ Shawmut Ave,2119.0,NaN,16,100.0,Shawmut Ave,Boston
1,Academy Homes I,2119.0,NaN,202,1592.0,Columbus Ave,Roxbury
2,Academy Homes II,2119.0,Y,236,2926.0,Washington St,Roxbury
3,Adams Court Phase A,2126.0,NaN,50,59.0,Msgr Patrick J Lydon Way,Boston
4,Adams Court Phase B,2126.0,NaN,45,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1440,xxxxxxxx Blue Hill Ave,2119.0,NaN,4,491.0,Blue Hill Avenue,Boston
1441,YouthBuild Ruskindale Rd,2136.0,NaN,1,1785.0,Columbus Ave #500,Roxbury
1442,YWCA Berkeley,2116.0,NaN,208,40.0,Berkeley St,Boston
1443,YWCA Boston Units,2116.0,NaN,115,140.0,Clarendon St,Boston


# Step 2. Overlay data between DBs

In [46]:
import re

def create_st_index(row):
    street_name = row["ST_NAME"]
    street_number = row["ST_NUM"]

    if street_name != street_name:
        return None

    street_name = street_name.lower()

    # Remove all non-alphanumeric characters
    street_name = re.sub(r'[^a-z0-9\s]', '', street_name)

    # Remove occurances of all these words from the street name
    remove_words = set(["ave","av", "st", "str", "street", "avenue"])
    street_name = ' '.join([word for word in street_name.split() if word not in remove_words])

    if not street_number or street_number != street_number:
        return street_name
    return f"{int(street_number)} {street_name}"

ah_df["join_idx"] = ah_df.apply(create_st_index, axis=1)
ir_df["join_idx"] = ir_df.apply(create_st_index, axis=1)

# Drop nans
ah_df.dropna(inplace=True, subset=["join_idx"])
ir_df.dropna(inplace=True, subset=["join_idx"])

# Set join_idx as a string
ah_df["join_idx"] = ah_df["join_idx"].astype(str)
ir_df["join_idx"] = ir_df["join_idx"].astype(str)

In [47]:
# Select only rows where join_idx is in the other dataframe
overlay_df = ah_df[ah_df["join_idx"].isin(ir_df["join_idx"])]
overlay_df.to_csv("../data/clean/result.csv", index=False)